### A Short Race

Lets see if there is any appreciable difference in the looping speed between a python loop and a C++ loop:

In [ ]:
import pyNUISANCE as pn
import pyProSelecta as pps

evs = pn.EventSource("dune_argon_sf_10mega.nuwro.pb.gz")
if not evs:
    print("Error: failed to open input file")

def enu_GeV(ev):
    bpart = pps.sel.Beam(ev,14)
    if bpart:
        return bpart.momentum().e() * 1E-3 # events are always with MeV momentum units
    return -0

In [ ]:
import time

In [ ]:
time_start = time.perf_counter()
for i, (ev, cvw) in enumerate(evs):
    enu_GeV(ev)
time_end = time.perf_counter()
pyloop_elapsed = (time_end-time_start)
print("pure python event loop took %.2fs" % pyloop_elapsed)

In [ ]:
time_start = time.perf_counter()
pn.FrameGen(evs).add_column("enu_GeV",enu_GeV).all()
time_end = time.perf_counter()
fg_pyfunc_elapsed = (time_end-time_start)
print("FrameGen with a python function took %.2fs" % fg_pyfunc_elapsed)

So that it actually slower than doing the loop in pure python! This is most likely due to the overheads of calling the python function on every event from the C++ side.

### ProSelecta

We can use ProSelecta to write and JIT compile C++ functions and then use the JITted functions to create new columns

In [ ]:
pps.load_text("""
double enu_GeV(HepMC3::GenEvent const &ev){
  auto bpart = ps::sel::Beam(ev,14);
  std::cout << "bpart: " << bool(bpart) << std::endl;
  if(bpart) {
    std::cout << "  mom: " << (bpart->momentum().e()*1E-3) << std::endl;
    return bpart->momentum().e()*1E-3;
  }
  return -0;
}
""")
enu_GeV_cpp = pps.project.enu_GeV

In [ ]:
time_start = time.perf_counter()
pn.FrameGen(evs).add_column("enu_GeV",enu_GeV_cpp).all()
time_end = time.perf_counter()
fg_ps_elapsed = (time_end-time_start)
print("FrameGen with ProSelecta took %.2fs" % fg_ps_elapsed)

In [ ]:
time_start = time.perf_counter()
pn.FrameGen(evs).all()
time_end = time.perf_counter()
fg_noop_elapsed = (time_end-time_start)
print("FrameGen with no-op took %.2fs" % fg_noop_elapsed)

So the majority of the time spent is reading the events off disk

In [ ]:
pyloop_elapsed -= fg_noop_elapsed
fg_pyfunc_elapsed -= fg_noop_elapsed
fg_ps_elapsed -= fg_noop_elapsed
print("IO Corrected: Pure Python %.2fs" % pyloop_elapsed)
print("IO Corrected: FrameGen with Python event processor %.2fs" % fg_pyfunc_elapsed)
print("IO Corrected: FrameGen with ProSelecta JIT compiled function %.2fs" % fg_ps_elapsed)